# Spaceship Titanic: TABM by Elias Ruud Aronsen

Project: [Spaceship Titanic](https://www.kaggle.com/competitions/spaceship-titanic/overview)

In this notebook, I will train a TABM model for the individual part of the project.

TABM Github repository can be found here: [TABM GitHub](https://github.com/yandex-research/tabm)

In [1]:
import pandas as pd
import random as rand
import torch
from torch import nn
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from src.tabm_reference import Model # This is our TABM model
from torch.utils.data import DataLoader, TensorDataset

# Read in preprocessed data, shown in 1-EDA-and-preprocessing.ipynb
train = pd.read_csv('data/processed_train.csv')
test = pd.read_csv('data/processed_test.csv')

df_Y = train['Transported']
df_X = train.drop(columns=['Transported'])

## TABM

In [2]:
df_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 46 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   RoomService                8693 non-null   float64
 1   FoodCourt                  8693 non-null   float64
 2   ShoppingMall               8693 non-null   float64
 3   Spa                        8693 non-null   float64
 4   VRDeck                     8693 non-null   float64
 5   Group                      8693 non-null   int64  
 6   Id                         8693 non-null   int64  
 7   Num                        8693 non-null   float64
 8   FamilySize                 8693 non-null   float64
 9   GroupSize                  8693 non-null   int64  
 10  HomePlanet_Earth           8693 non-null   float64
 11  HomePlanet_Europa          8693 non-null   float64
 12  HomePlanet_Mars            8693 non-null   float64
 13  HomePlanet_nan             8693 non-null   float

In [ ]:
#### MODEL DEFINITION AND SETUP

train_x, val_x, train_y, val_y = train_test_split( df_X, df_Y, test_size=0.2, random_state=42, stratify=df_Y) # split our dataset # stratify makes sure train and test have same class distribution

# we turn data int tensors as we will be using pytorch.
train_x_tensor = torch.tensor(train_x.values, dtype=torch.float32)
val_x_tensor = torch.tensor(val_x.values, dtype=torch.float32)
train_y_tensor = torch.tensor(train_y.values, dtype=torch.long)
val_y_tensor = torch.tensor(val_y.values, dtype=torch.long)

# create the tensor datasets
train_dataset = TensorDataset(train_x_tensor, train_y_tensor)
val_dataset = TensorDataset(val_x_tensor, val_y_tensor)

# create dataloaders for batches, speeds up training, self regularization, more stable updates etc.
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512)


### Model setup
num_features = train_x_tensor.shape[1] # gives the amount of features, which is 46

# setup function which defines our model, loss function and optimizer with given parameters.
def setup( #default values
    n_blocks=5, 
    d_block=256, 
    k=3, 
    dropout=0.1,
    activation='ReLU', 
    lr=1e-3, 
    weight_decay=1e-4): # default settings

    model = Model(
        n_num_features=num_features,  # number of numeric features
        cat_cardinalities=[],  # list with number of categories for each categorical feature, empty since we have converted everything to numeric
        n_classes=2 ,  # number of output classes, 2 for binary
        
        backbone=dict(  # structure of the underlying MLPs
            type='MLP',  # simple feedforward network
            n_blocks=n_blocks,  # number of layers (depth)
            d_block=d_block,  # width of each layer (hidden size)
            dropout=dropout,  # dropout between layers for regularization
            activation=activation,  # activation function like ReLU or GELU 
        ),
    
        bins=None,  # used if we had numerical binning, we used one hot encoding for age instead
        num_embeddings=None,  # used if we had embedding of numeric features, we dont nees this
        arch_type='tabm',  # architecture type, "tabm" for TabM model
        k=k,  # number of experts the model can choose from at each layer
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay) # we tune learning rate for speed and stability and weight decay for regularization
    loss_function = nn.CrossEntropyLoss() # cross entropy is standard for binary classification

    return model, optimizer, loss_function


In [ ]:
##### Training the TABM MODEL
# training function
def run_tabm(
    n_blocks=5, 
    d_block=256, 
    k=3, 
    dropout=0.1,
    activation='ReLU',
    lr=1e-3, 
    weight_decay=1e-4, 
    n_epochs=200, 
    patience=20 ):

    # define our model optimizer and loss function parameters wiht given parameters
    model, optimizer, loss_function = setup(n_blocks=n_blocks, d_block=d_block, k=k, dropout=dropout, activation=activation, lr=lr, weight_decay=weight_decay)

    best_val_loss = float('inf') # tracks current best validation loss reached
    patience_counter = 0 # goes up when the model steps into a worse val loss.
    
    #Training loop with validation monitoring and early stopping.
    for epoch in range(1, n_epochs + 1):
        #Training
        model.train()
        train_losses = [] # these will hold our losses for the batch, at the end of epoch take the average loss for each batch
        train_preds_all = [] # stored predictions for each batch, for calculation of accuracy later
        train_targets_all = [] # true labels for each batch, for calculation of accuracy later

        for xb, yb in train_loader: # loading in batches
            optimizer.zero_grad()
            outputs = model(xb, None) # forward propogation
            loss = loss_function(outputs.mean(dim=1), yb) # calculate loss
            loss.backward() # backward propegation
            optimizer.step()

            train_losses.append(loss.item()) # saves current batch loss
            train_preds_all.append(outputs.mean(dim=1).argmax(dim=1)) # we take the average across k preds, pick the predicted class and store it for each batch
            train_targets_all.append(yb) # stores the true labels
            
        # calculating accuracy for epoch
        train_preds_all = torch.cat(train_preds_all) # stich all batch preds in one tensore
        train_targets_all = torch.cat(train_targets_all) # 	same for the true labels
        train_acc = accuracy_score(train_targets_all.cpu(), train_preds_all.cpu()) # computes accuracy

        # Validation, mostly these same just no training 
        model.eval()
        val_losses = [] # these does the same thing as train_losses, train_preds_all and train_targets_all
        val_preds_all = []
        val_targets_all = []

        with torch.no_grad():
            for xb, yb in val_loader:
                val_outputs = model(xb, None)
                val_loss = loss_function(val_outputs.mean(dim=1), yb)

                val_losses.append(val_loss.item())
                val_preds_all.append(val_outputs.mean(dim=1).argmax(dim=1))
                val_targets_all.append(yb)

        # calculating validation accuracy
        val_preds_all = torch.cat(val_preds_all)
        val_targets_all = torch.cat(val_targets_all)
        val_acc = accuracy_score(val_targets_all.cpu(), val_preds_all.cpu())
        val_loss_epoch = sum(val_losses) / len(val_losses)

        print(f"Epoch {epoch}: Train Loss={sum(train_losses)/len(train_losses):.4f}, Train Acc={train_acc:.4f} ||| Val Loss={val_loss_epoch:.4f}, Val Acc={val_acc:.4f}")

        # early stopping check
        if val_loss_epoch < best_val_loss:
            best_val_loss = val_loss_epoch
            patience_counter = 0 # we reset counter each time we find better
            best_model_state = model.state_dict() # save state
        else: # increase patience if new is not better
            patience_counter += 1
            if patience_counter >= patience: # stop training if model has not produced a better val loss in the defined amount of epochs
                print(f"Early stopping triggered at epoch {epoch}")
                break
            
    # Load back the best found model
    model.load_state_dict(best_model_state)
    return model
    

### Tuning

In [ ]:
import optuna

# how many hyperparameter combinations to try
n_trials = 50

# we define the objective function that Optuna will try to maximize
def objective(trial):
    # suggest a value for each hyperparameter
    ## The parameter space based recommendations from: https://github.com/yandex-research/tabm/tree/main
    params = {
        'n_blocks': trial.suggest_int('n_blocks', 2, 6),
        'd_block': trial.suggest_categorical('d_block', [128, 256, 512]),
        'k': trial.suggest_int('k', 4, 12),
        'dropout': trial.suggest_float('dropout', 0.1, 0.5),
        'activation': trial.suggest_categorical('activation', ['ReLU', 'GELU', 'LeakyReLU']),
        'lr': trial.suggest_loguniform('lr', 1e-4, 5e-2),
        'weight_decay': trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)
    }
    
    print(f"\n Trial {trial.number+1} with params: {params}")

    # train the model with the chosen parameters
    model = run_tabm(**params)

    # after we train, we evaluate the final val accuracy
    model.eval()  # set model to eval mode. turns off dropout etc.

    val_preds = []    # predictions
    val_targets = []  # targets

    with torch.no_grad():  # disables gradient tracking, faster
        for xb, yb in val_loader:  # go over val batches
            outputs = model(xb, None)  # forward pass
            preds = outputs.mean(dim=1).argmax(dim=1)  # ensemble average, then pick predicted class
            val_preds.append(preds)
            val_targets.append(yb)

    val_preds = torch.cat(val_preds)
    val_targets = torch.cat(val_targets)

    val_acc = accuracy_score(val_targets.cpu(), val_preds.cpu())  # accuracy calculation
    print(f"Validation Accuracy: {val_acc:.4f}")

    return val_acc  # optuna will try to maximize this


# create a study object
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=n_trials)

# save all results
results = [(t.params, t.value) for t in study.trials]

# best trial found
print(f"\nBest Trial Params: {study.best_params}")
print(f"Best Validation Accuracy: {study.best_value:.4f}")

c:\Users\elias\miniconda3\envs\TABM\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-04-27 16:52:35,433] A new study created in memory with name: no-name-60be0ac5-c2a0-4e78-b558-45f50a391fd7



 Trial 1 with params: {'n_blocks': 2, 'd_block': 128, 'k': 8, 'dropout': 0.4226779073013317, 'activation': 'GELU', 'lr': 0.0010017908773783256, 'weight_decay': 5.48391092738397e-06}


C:\Users\elias\AppData\Local\Temp\ipykernel_21576\1656162094.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lr': trial.suggest_loguniform('lr', 1e-4, 5e-2),                   # learning rate (log scale)
C:\Users\elias\AppData\Local\Temp\ipykernel_21576\1656162094.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'weight_decay': trial.suggest_loguniform('weight_decay', 1e-6, 1e-2) # optimizer regularization


Epoch 1: Train Loss=8.2198, Train Acc=0.6445 ||| Val Loss=1.2242, Val Acc=0.7550
Epoch 2: Train Loss=2.2198, Train Acc=0.6622 ||| Val Loss=0.6161, Val Acc=0.7608
Epoch 3: Train Loss=1.0650, Train Acc=0.6624 ||| Val Loss=0.6124, Val Acc=0.6665
Epoch 4: Train Loss=0.7555, Train Acc=0.6695 ||| Val Loss=0.6185, Val Acc=0.6653
Epoch 5: Train Loss=0.6752, Train Acc=0.6800 ||| Val Loss=0.6124, Val Acc=0.6665
Epoch 6: Train Loss=0.6196, Train Acc=0.6926 ||| Val Loss=0.5936, Val Acc=0.6809
Epoch 7: Train Loss=0.5952, Train Acc=0.7123 ||| Val Loss=0.5979, Val Acc=0.6711
Epoch 8: Train Loss=0.5939, Train Acc=0.7094 ||| Val Loss=0.5940, Val Acc=0.6820
Epoch 9: Train Loss=0.5802, Train Acc=0.7181 ||| Val Loss=0.5797, Val Acc=0.6855
Epoch 10: Train Loss=0.5698, Train Acc=0.7216 ||| Val Loss=0.5812, Val Acc=0.6889
Epoch 11: Train Loss=0.5579, Train Acc=0.7302 ||| Val Loss=0.5676, Val Acc=0.7234
Epoch 12: Train Loss=0.5540, Train Acc=0.7321 ||| Val Loss=0.5655, Val Acc=0.6952
Epoch 13: Train Loss=0.54

[I 2025-04-27 16:53:17,548] Trial 0 finished with value: 0.7975848188614146 and parameters: {'n_blocks': 2, 'd_block': 128, 'k': 8, 'dropout': 0.4226779073013317, 'activation': 'GELU', 'lr': 0.0010017908773783256, 'weight_decay': 5.48391092738397e-06}. Best is trial 0 with value: 0.7975848188614146.


...

Epoch 87: Train Loss=0.4329, Train Acc=0.7980 ||| Val Loss=0.4451, Val Acc=0.7913
Early stopping triggered at epoch 87
Validation Accuracy: 0.7913

 Trial 50 with params: {'n_blocks': 5, 'd_block': 128, 'k': 9, 'dropout': 0.34994928353452004, 'activation': 'ReLU', 'lr': 0.006338294096152983, 'weight_decay': 0.001276334155072742}


C:\Users\elias\AppData\Local\Temp\ipykernel_21576\1656162094.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lr': trial.suggest_loguniform('lr', 1e-4, 5e-2),                   # learning rate (log scale)
C:\Users\elias\AppData\Local\Temp\ipykernel_21576\1656162094.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'weight_decay': trial.suggest_loguniform('weight_decay', 1e-6, 1e-2) # optimizer regularization


Epoch 1: Train Loss=0.7626, Train Acc=0.6923 ||| Val Loss=0.5103, Val Acc=0.7815
Epoch 2: Train Loss=0.5301, Train Acc=0.7752 ||| Val Loss=0.4980, Val Acc=0.7913
Epoch 3: Train Loss=0.5173, Train Acc=0.7791 ||| Val Loss=0.5021, Val Acc=0.7907
Epoch 4: Train Loss=0.5181, Train Acc=0.7787 ||| Val Loss=0.4901, Val Acc=0.7907
Epoch 5: Train Loss=0.5172, Train Acc=0.7791 ||| Val Loss=0.4970, Val Acc=0.7855
Epoch 6: Train Loss=0.5246, Train Acc=0.7751 ||| Val Loss=0.4902, Val Acc=0.7913
Epoch 7: Train Loss=0.5176, Train Acc=0.7787 ||| Val Loss=0.4977, Val Acc=0.7832
Epoch 8: Train Loss=0.5210, Train Acc=0.7811 ||| Val Loss=0.5178, Val Acc=0.7872
Epoch 9: Train Loss=0.5189, Train Acc=0.7771 ||| Val Loss=0.4955, Val Acc=0.7884
Epoch 10: Train Loss=0.5169, Train Acc=0.7785 ||| Val Loss=0.4996, Val Acc=0.7867
Epoch 11: Train Loss=0.5159, Train Acc=0.7775 ||| Val Loss=0.4863, Val Acc=0.7878
Epoch 12: Train Loss=0.5185, Train Acc=0.7797 ||| Val Loss=0.4898, Val Acc=0.7878
Epoch 13: Train Loss=0.51

[I 2025-04-27 17:30:17,617] Trial 49 finished with value: 0.7924094307073031 and parameters: {'n_blocks': 5, 'd_block': 128, 'k': 9, 'dropout': 0.34994928353452004, 'activation': 'ReLU', 'lr': 0.006338294096152983, 'weight_decay': 0.001276334155072742}. Best is trial 30 with value: 0.8125359401955147.


Epoch 59: Train Loss=0.5121, Train Acc=0.7856 ||| Val Loss=0.4849, Val Acc=0.7924
Early stopping triggered at epoch 59
Validation Accuracy: 0.7924

Best Trial Params: {'n_blocks': 5, 'd_block': 128, 'k': 10, 'dropout': 0.27284433651928997, 'activation': 'ReLU', 'lr': 0.002029456931526584, 'weight_decay': 3.9033694841500456e-06}
Best Validation Accuracy: 0.8125


In [ ]:
# sort the results based on performance and show best parameter combinations
best_params = sorted(results, key=lambda x: x[1], reverse=True)

print("\nTop 5 parameter combinations:") # print the top 5 parameter combinations
for i, (params, acc) in enumerate(best_params[:5], start=1):
    print(f"{i}. Val Accuracy: {acc:.4f} | Params: {params}")


Top 5 parameter combinations:
1. Val Accuracy: 0.8125 | Params: {'n_blocks': 5, 'd_block': 128, 'k': 10, 'dropout': 0.27284433651928997, 'activation': 'ReLU', 'lr': 0.002029456931526584, 'weight_decay': 3.9033694841500456e-06}
2. Val Accuracy: 0.8097 | Params: {'n_blocks': 4, 'd_block': 128, 'k': 10, 'dropout': 0.15984630203331762, 'activation': 'ReLU', 'lr': 0.0014769364639454208, 'weight_decay': 9.200748737757358e-06}
3. Val Accuracy: 0.8068 | Params: {'n_blocks': 5, 'd_block': 128, 'k': 9, 'dropout': 0.22242292358131166, 'activation': 'ReLU', 'lr': 0.004070611397840372, 'weight_decay': 4.672432428590436e-06}
4. Val Accuracy: 0.8068 | Params: {'n_blocks': 5, 'd_block': 128, 'k': 9, 'dropout': 0.24359651545595912, 'activation': 'ReLU', 'lr': 0.005323721854063274, 'weight_decay': 3.296597682039308e-05}
5. Val Accuracy: 0.8062 | Params: {'n_blocks': 5, 'd_block': 128, 'k': 10, 'dropout': 0.2256010105258364, 'activation': 'ReLU', 'lr': 0.003487671318826522, 'weight_decay': 6.99820250497

## Results

- After 30 trials the best accuracy attained was 0.8125, which is on par/slightly worse than a tuned CatModel, but better than most of the ensemble models, but not quite as good as I expected, but we still have not trained the model on the full dataset which will likely increase it's accuracy.

From here we train this model in preperation for the submssion on the full training set, and generate our predictions on the given test data and submit.

In [ ]:
from sklearn.model_selection import StratifiedKFold

best = best_params[0][0] # get our best found parameters
print("Our parameters: ", best)

# prepare the full training dataset
X_tensor = torch.tensor(df_X.values, dtype=torch.float32)
Y_tensor = torch.tensor(df_Y.values, dtype=torch.long)

# The cross-validation setup
n_splits = 5 # amount of folds
strat_k_fold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

cv_scores = [] # contains our

# the k fold loop
for fold_idx, (train_idx, val_idx) in enumerate(strat_k_fold.split(X_tensor, Y_tensor)):
    print(f"\n ||| Fold number: {fold_idx + 1} / {n_splits} |||")

    # we split into this folds training and validations subsets
    X_train_fold = X_tensor[train_idx]
    y_train_fold = Y_tensor[train_idx]
    X_val_fold = X_tensor[val_idx]
    y_val_fold = Y_tensor[val_idx]

    # create the DataLoaders
    train_dataset = TensorDataset(X_train_fold, y_train_fold)
    val_dataset = TensorDataset(X_val_fold, y_val_fold)

    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True) # larger batch sizes to combat overfitting
    val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)
    # evaluate on validation set
    
    # train the model for this fold
    fold_model = run_tabm(**best, n_epochs=400)
    
    fold_model.eval()  # turn off dropout, etc.

    val_preds = [] # predictions
    val_targets = [] # true labels

    with torch.no_grad():  # disables gradient tracking
        for xb, yb in val_loader:  # use your validation loader
            outputs = fold_model(xb, None)
            preds = outputs.mean(dim=1).argmax(dim=1) # generate final prediction from the k experts
            val_preds.append(preds)
            val_targets.append(yb)

    val_preds = torch.cat(val_preds)
    val_targets = torch.cat(val_targets)

    # calculate this folds validation accuracy
    val_acc = accuracy_score(val_targets.cpu(), val_preds.cpu())
    print(f"Fold {fold_idx + 1} Validation Accuracy: {val_acc:.4f}")
    cv_scores.append(val_acc)


print("\n--- Cross-Validation Results ---")
print(f"Mean Validation Accuracy: {sum(cv_scores)/len(cv_scores):.4f}")
print(f"Validation Accuracies for each fold: {cv_scores}")

Our parameters:  {'n_blocks': 5, 'd_block': 128, 'k': 10, 'dropout': 0.27284433651928997, 'activation': 'ReLU', 'lr': 0.002029456931526584, 'weight_decay': 3.9033694841500456e-06}

 ||| Fold number: 1 / 5 |||
Epoch 1: Train Loss=0.9427, Train Acc=0.5552 ||| Val Loss=0.6188, Val Acc=0.7694
Epoch 2: Train Loss=0.6125, Train Acc=0.6762 ||| Val Loss=0.5430, Val Acc=0.7499
Epoch 3: Train Loss=0.5556, Train Acc=0.7327 ||| Val Loss=0.5254, Val Acc=0.7924
Epoch 4: Train Loss=0.5386, Train Acc=0.7498 ||| Val Loss=0.5030, Val Acc=0.7964
Epoch 5: Train Loss=0.5270, Train Acc=0.7529 ||| Val Loss=0.5024, Val Acc=0.7918
Epoch 6: Train Loss=0.5208, Train Acc=0.7714 ||| Val Loss=0.4957, Val Acc=0.7999
Epoch 7: Train Loss=0.5182, Train Acc=0.7741 ||| Val Loss=0.4906, Val Acc=0.7959
Epoch 8: Train Loss=0.5059, Train Acc=0.7819 ||| Val Loss=0.4880, Val Acc=0.7970
Epoch 9: Train Loss=0.5072, Train Acc=0.7787 ||| Val Loss=0.5041, Val Acc=0.7941
Epoch 10: Train Loss=0.5073, Train Acc=0.7788 ||| Val Loss=0.4

In [18]:
print("\n ||| Training final model on full dataset |||")

# use full training dataset on the model
full_dataset = TensorDataset(X_tensor, Y_tensor)
train_loader = DataLoader(full_dataset, batch_size=256, shuffle=True) #  global train_loader variable used inside run_tabm

# training final model
final_model = run_tabm(**best, n_epochs=400)

# evaluate validation accuracy
final_model.eval()  # turn off dropout, etc.

val_preds = [] # predictions
val_targets = [] # true labels

with torch.no_grad():  # disables gradient tracking
    for xb, yb in val_loader:  # use your validation loader
        outputs = final_model(xb, None)
        preds = outputs.mean(dim=1).argmax(dim=1)
        val_preds.append(preds)
        val_targets.append(yb)

val_preds = torch.cat(val_preds)
val_targets = torch.cat(val_targets)

# calculate final validation accuracy
final_val_acc = accuracy_score(val_targets.cpu(), val_preds.cpu())
print(f"Final Validation Accuracy: {final_val_acc:.4f}")


 ||| Training final model on full dataset |||
Epoch 1: Train Loss=0.9117, Train Acc=0.5651 ||| Val Loss=0.6034, Val Acc=0.7468
Epoch 2: Train Loss=0.5901, Train Acc=0.6963 ||| Val Loss=0.5344, Val Acc=0.7589
Epoch 3: Train Loss=0.5538, Train Acc=0.7438 ||| Val Loss=0.5098, Val Acc=0.7779
Epoch 4: Train Loss=0.5356, Train Acc=0.7611 ||| Val Loss=0.4949, Val Acc=0.7722
Epoch 5: Train Loss=0.5207, Train Acc=0.7700 ||| Val Loss=0.4986, Val Acc=0.7768
Epoch 6: Train Loss=0.5130, Train Acc=0.7741 ||| Val Loss=0.4950, Val Acc=0.7791
Epoch 7: Train Loss=0.5068, Train Acc=0.7790 ||| Val Loss=0.4861, Val Acc=0.7756
Epoch 8: Train Loss=0.5034, Train Acc=0.7798 ||| Val Loss=0.4840, Val Acc=0.7762
Epoch 9: Train Loss=0.4945, Train Acc=0.7858 ||| Val Loss=0.4785, Val Acc=0.7768
Epoch 10: Train Loss=0.4998, Train Acc=0.7822 ||| Val Loss=0.4887, Val Acc=0.7831
Epoch 11: Train Loss=0.4923, Train Acc=0.7852 ||| Val Loss=0.4795, Val Acc=0.7779
Epoch 12: Train Loss=0.4833, Train Acc=0.7873 ||| Val Loss=0

#### Making the submission

In [19]:

# convert test to tensor so that we can feed it into model
test_x_tensor = torch.tensor(test.values, dtype=torch.float32)

# Predicting
final_model.eval()
with torch.no_grad():
    test_outputs = final_model(test_x_tensor, None)  # generate predictions for the k models
    test_preds = test_outputs.mean(dim=1).argmax(dim=1)  # we take the average prediction over the k ensemble models

# we get back the passengerIds (removed in encoding)
test_ids = pd.read_csv('data/test.csv')['PassengerId']

#  setup the submission df
final_submission_tabm = pd.DataFrame({
    'PassengerId': test_ids,
    'Transported': test_preds.cpu().numpy().astype(bool)  # convert predictions to bool True/False
})

# saves the submission df to csv. Ready for kaggle.
final_submission_tabm.to_csv('submissions/final_submission_tabm.csv', index=False)

## Final Result

- The final model achieved a validation accuracy of **0.8470**, which is a strong result during training.
- However, the final Kaggle leaderboard score for the TABM model was **0.79541**, indicating signs of overfitting to the training data.
- The five-fold cross-validation gave a mean validation accuracy of **0.7927**, suggesting that the true generalization performance of the model is slightly lower than the single validation split suggested.

I hypothesize that the model's complexity — and the high capacity of MLP architectures in general — made it prone to overfitting given the relatively small size of the Spaceship Titanic dataset.  
Despite the regularization strategies used (dropout, weight decay, batch normalization), the model likely still overfit subtle patterns in the training data that did not generalize well to unseen test data.
